In [1]:
import numpy as np
import xarray as xr
import pandas as pd
from netCDF4 import Dataset
import os,datetime,sys,fnmatch
import time
import math
import dask.array as da
import graphviz

In [2]:
def read_MODIS_level2_data(MOD06_file,MOD03_file):
    print(MOD06_file)
    print(MOD03_file)
    print('reading the cloud mask from MOD06_L2 product')
    myd06 = Dataset(MOD06_file, "r")
    CM = myd06.variables["Cloud_Mask_1km"][:,:,:] # Reading Specific Variable 'Cloud_Mask_1km'.
    CM   = (np.array(CM[:,:,0],dtype='byte') & 0b00000110) >>1
    CM = np.array(CM).byteswap().newbyteorder()
    #CM = np.ravel(CM) # Changing The Shape Of The Variable 'Longitude'.
    print('level-2 cloud mask array shape',CM.shape)

    myd03 = Dataset(MOD03_file, "r")
    print('reading the lat-lon from MOD03 product')
    latitude = myd03.variables["Latitude"][:,:] # Reading Specific Variable 'Latitude'.
    #latitude = np.ravel(latitude) # Changing The Shape Of The Variable 'Latitude'.
    latitude = np.array(latitude).byteswap().newbyteorder() # Addressing Byteswap For Big Endian Error.
    longitude = myd03.variables["Longitude"][:,:] # Reading Specific Variable 'Longitude'.
    #longitude = np.ravel(longitude) # Changing The Shape Of The Variable 'Longitude'.
    longitude = np.array(longitude).byteswap().newbyteorder() # Addressing Byteswap For Big Endian Error.
    print('level-2 lat-lon array shape',latitude.shape)

    return latitude,longitude,CM

In [3]:
def value_locate(refx, x):
    refx = np.array(refx)
    x = np.array(x)
    loc = np.zeros(len(x), dtype='int')

    for i in range(len(x)):
        ix = x[i]
        ind = ((refx - ix) <= 0).nonzero()[0]
        if len(ind) == 0:
            loc[i] = -1
        else: loc[i] = ind[-1]

    return loc

def division(n, d):

    div = np.zeros(len(d))
    for i in range(len(d)):
        if d[i] >0:
          div[i]=n[i]/d[i]
        else: div[i]=None 

    return div

In [4]:
!set MOD03_PATH = /home/jovyan/Files/

In [5]:
%env MOD03_PATH=/home/jovyan/Files/
%env MOD06_PATH=/home/jovyan/Files/

env: MOD03_PATH=/home/jovyan/Files/
env: MOD06_PATH=/home/jovyan/Files/


In [6]:
%env MOD03_PATH

'/home/jovyan/Files/'

In [7]:
%env MOD06_PATH

'/home/jovyan/Files/'

In [8]:
#MOD03_path = '/Users/saviosebastian/Documents/Project/CMAC/HDFFiles/'
#MOD06_path = '/Users/saviosebastian/Documents/Project/CMAC/HDFFiles/'
MOD03_path = %env MOD03_PATH
MOD06_path = %env MOD06_PATH
satellite = 'Aqua'

yr = [2008]
mn = [1] #np.arange(1,13)  #[1]
dy = [1] #np.arange(1,32) # [1] #np.arange(1,31)
# latitude and longtitude boundaries of level-3 grid
lat_bnd = np.arange(-90,91,1)
lon_bnd = np.arange(-180,180,1)
nlat = 180
nlon = 360

TOT_pix      = np.zeros(nlat*nlon)
CLD_pix      = np.zeros(nlat*nlon)

In [9]:
MOD03_fp = 'MYD03.A*.hdf'
MOD06_fp = 'MYD06_L2.A*.hdf'
MOD03_fn, MOD06_fn =[],[]
#MOD03_fn2, MOD06_fn2 =[],[]
MOD03_fn2, MOD06_fn2 =[],[]
for MOD06_flist in  os.listdir(MOD06_path):
    if fnmatch.fnmatch(MOD06_flist, MOD06_fp):
        MOD06_fn = MOD06_flist
        MOD06_fn2.append(MOD06_flist)
        print(MOD06_fn)
for MOD03_flist in  os.listdir(MOD03_path):
    if fnmatch.fnmatch(MOD03_flist, MOD03_fp):
        MOD03_fn = MOD03_flist
        MOD03_fn2.append(MOD03_flist)
        print(MOD03_fn)
if MOD03_fn and MOD06_fn:
    # if both MOD06 and MOD03 products are in the directory
    print('reading level 2 geolocation and cloud data')
    #print(MOD06_fn)
    #print(MOD03_fn)
    Lat,Lon,CM = read_MODIS_level2_data(MOD06_path+MOD06_fn,MOD03_path+MOD03_fn)
    

MYD06_L2.A2008001.0005.006.2013341193207.hdf
MYD06_L2.A2008001.0010.006.2013341192125.hdf
MYD06_L2.A2008001.0000.006.2013341193524.hdf
MYD03.A2008001.0010.006.2012066122416.hdf
MYD03.A2008001.0000.006.2012066122450.hdf
MYD03.A2008001.0005.006.2012066122516.hdf
reading level 2 geolocation and cloud data
/home/jovyan/Files/MYD06_L2.A2008001.0000.006.2013341193524.hdf
/home/jovyan/Files/MYD03.A2008001.0005.006.2012066122516.hdf
reading the cloud mask from MOD06_L2 product
level-2 cloud mask array shape (2030, 1354)
reading the lat-lon from MOD03 product
level-2 lat-lon array shape (2030, 1354)


In [10]:
MOD03_fn2

['MYD03.A2008001.0010.006.2012066122416.hdf',
 'MYD03.A2008001.0000.006.2012066122450.hdf',
 'MYD03.A2008001.0005.006.2012066122516.hdf']

In [11]:
MOD06_flist

'MYD06_L2.A2008001.0000.006.2013341193524.hdf'

In [12]:
MOD06_fn2

['MYD06_L2.A2008001.0005.006.2013341193207.hdf',
 'MYD06_L2.A2008001.0010.006.2013341192125.hdf',
 'MYD06_L2.A2008001.0000.006.2013341193524.hdf']

In [13]:
MOD06_path

'/home/jovyan/Files/'

In [14]:
%time
Lat

CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 6.68 µs


array([[52.160187, 52.159992, 52.15975 , ..., 47.556084, 47.538456,
        47.520664],
       [52.142014, 52.141872, 52.141685, ..., 47.539726, 47.52207 ,
        47.504246],
       [52.123837, 52.123753, 52.12362 , ..., 47.523388, 47.505676,
        47.487812],
       ...,
       [34.20459 , 34.202507, 34.20042 , ..., 30.682018, 30.670088,
        30.658052],
       [34.186485, 34.18446 , 34.18243 , ..., 30.664764, 30.652782,
        30.640692],
       [34.168385, 34.166416, 34.16444 , ..., 30.647491, 30.635452,
        30.623335]], dtype=float32)

In [15]:
Lat.shape

(2030, 1354)

In [16]:
Lon.shape

(2030, 1354)

In [17]:
CM.shape

(2030, 1354)

In [18]:
#for items in MOD06_fn2:

In [84]:
myd06_name = '/home/jovyan/Files/'
#cm = np.array(2)
cm = np.zeros((2030,1354), dtype=np.float32)
#cm = np.empty([2, 2])
#cm1 = np.array(2)
cm1 = np.zeros((2030,1354), dtype=np.float32)
#cm1 = np.empty([2, 2])
#cmr = np.array(2)
cmr = np.zeros((2030,1354), dtype=np.float32)
#cmr = np.empty([2, 2])
for MOD06_file in MOD06_fn2:
    MOD06_file2 = myd06_name + MOD06_file
    myd06 = Dataset(MOD06_file2, "r")
    CM = myd06.variables["Cloud_Mask_1km"][:,:,:]# Reading Specific Variable 'Cloud_Mask_1km'.
    CMr = myd06.variables["Cloud_Mask_1km"][:,:,0]
    CM1   = (np.array(CM[:,:,0],dtype='byte') & 0b00000001)
    CM   = (np.array(CM[:,:,0],dtype='byte') & 0b00000110) >>1
    CM = np.array(CM).byteswap().newbyteorder()
    cm = np.dstack((cm,CM))
    cm1 = np.dstack((cm1,CM1))
    cmr = np.dstack((cmr,CMr))
    #CM = np.ravel(CM) # Changing The Shape Of The Variable 'Longitude'.
    #ar.append(CM)
    #cm = np.append(cm, CM)
    #cm = np.concatenate((cm,CM), axis=0)
    #cm = np.column_stack([cm,CM])
    #cm1 = np.append(cm1, CM1)
    #cmr = np.append(cmr, CMr)
    #ar.insert(CM)
    print('level-2 cloud mask array shape',CM.shape)
    
    

level-2 cloud mask array shape (2030, 1354)
level-2 cloud mask array shape (2030, 1354)
level-2 cloud mask array shape (2030, 1354)


In [20]:
cmtemp = np.zeros((2030,1354), dtype=np.float32)
cmtemp

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [85]:
cm.shape

(2030, 1354, 4)

In [86]:
cm

array([[[0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        ...,
        [0., 0., 1., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 1., 0., 0.],
        [0., 1., 0., 1.],
        [0., 1., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 1., 0., 0.],
        [0., 1., 0., 1.],
        [0., 2., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       ...,

       [[0., 0., 3., 1.],
        [0., 0., 3., 1.],
        [0., 0., 3., 1.],
        ...,
        [0., 0., 3., 0.],
        [0., 0., 3., 0.],
        [0., 0., 3., 0.]],

       [[0., 0., 3., 1.],
        [0., 2., 3., 1.],
        [0., 0., 3., 0.],
        ...,
        [0., 0., 3., 0.],
        [0., 0., 3., 0.],
        [0., 0., 3., 0.]],

       [[0., 1., 3., 1.],
        [0., 2., 3., 1.],
        [0., 2., 3., 0.],
        ...,
        [0., 0., 1., 0.],
        [0., 0.

In [23]:
cm1.shape

(2030, 1354, 4)

In [24]:
MOD06_fn2

['MYD06_L2.A2008001.0005.006.2013341193207.hdf',
 'MYD06_L2.A2008001.0010.006.2013341192125.hdf',
 'MYD06_L2.A2008001.0000.006.2013341193524.hdf']

In [25]:
cm1.shape

(2030, 1354, 4)

In [26]:
cm

array([[[0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        ...,
        [0., 0., 1., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 1., 0., 0.],
        [0., 1., 0., 1.],
        [0., 1., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 1., 0., 0.],
        [0., 1., 0., 1.],
        [0., 2., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       ...,

       [[0., 0., 3., 1.],
        [0., 0., 3., 1.],
        [0., 0., 3., 1.],
        ...,
        [0., 0., 3., 0.],
        [0., 0., 3., 0.],
        [0., 0., 3., 0.]],

       [[0., 0., 3., 1.],
        [0., 2., 3., 1.],
        [0., 0., 3., 0.],
        ...,
        [0., 0., 3., 0.],
        [0., 0., 3., 0.],
        [0., 0., 3., 0.]],

       [[0., 1., 3., 1.],
        [0., 2., 3., 1.],
        [0., 2., 3., 0.],
        ...,
        [0., 0., 1., 0.],
        [0., 0.

In [87]:
cm = da.from_array(cm, chunks =(2030,1354,1))
cm1 = da.from_array(cm1, chunks =(2030,1354,1))
cmr = da.from_array(cmr, chunks =(2030,1354,1))

In [28]:
#cm = da.from_array(CM, chunks =(1015,677))

In [29]:
cm.compute()

array([[[0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        ...,
        [0., 0., 1., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 1., 0., 0.],
        [0., 1., 0., 1.],
        [0., 1., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 1., 0., 0.],
        [0., 1., 0., 1.],
        [0., 2., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       ...,

       [[0., 0., 3., 1.],
        [0., 0., 3., 1.],
        [0., 0., 3., 1.],
        ...,
        [0., 0., 3., 0.],
        [0., 0., 3., 0.],
        [0., 0., 3., 0.]],

       [[0., 0., 3., 1.],
        [0., 2., 3., 1.],
        [0., 0., 3., 0.],
        ...,
        [0., 0., 3., 0.],
        [0., 0., 3., 0.],
        [0., 0., 3., 0.]],

       [[0., 1., 3., 1.],
        [0., 2., 3., 1.],
        [0., 2., 3., 0.],
        ...,
        [0., 0., 1., 0.],
        [0., 0.

In [30]:
cmtemp = da.from_array(cm, chunks =(500,500,1))
cmtemp.compute()

array([[[0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        ...,
        [0., 0., 1., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 1., 0., 0.],
        [0., 1., 0., 1.],
        [0., 1., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 1., 0., 0.],
        [0., 1., 0., 1.],
        [0., 2., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       ...,

       [[0., 0., 3., 1.],
        [0., 0., 3., 1.],
        [0., 0., 3., 1.],
        ...,
        [0., 0., 3., 0.],
        [0., 0., 3., 0.],
        [0., 0., 3., 0.]],

       [[0., 0., 3., 1.],
        [0., 2., 3., 1.],
        [0., 0., 3., 0.],
        ...,
        [0., 0., 3., 0.],
        [0., 0., 3., 0.],
        [0., 0., 3., 0.]],

       [[0., 1., 3., 1.],
        [0., 2., 3., 1.],
        [0., 2., 3., 0.],
        ...,
        [0., 0., 1., 0.],
        [0., 0.

In [31]:
cmtemp.size

10994480

In [32]:
cmtemp.shape

(2030, 1354, 4)

In [33]:
cm.chunks

((2030,), (1354,), (1, 1, 1, 1))

In [35]:
myd03_name = '/home/jovyan/Files/'
#lat = np.array(2)
lat = np.zeros((2030,1354), dtype=np.float32)
#lon = np.array(2)
lon = np.zeros((2030,1354), dtype=np.float32)
for MOD03_file in MOD03_fn2:
    MOD03_file2 = myd03_name + MOD03_file
    myd03 = Dataset(MOD03_file2, "r")
    latitude = myd03.variables["Latitude"][:,:] # Reading Specific Variable 'Latitude'.
    lat = np.dstack((lat,latitude))
    #latitude = np.ravel(latitude) # Changing The Shape Of The Variable 'Latitude'.
    #latitude = np.array(latitude).byteswap().newbyteorder() # Addressing Byteswap For Big Endian Error.
    #lat = np.append(lat, latitude)
    print('Latitude Shape Is: ',lat.shape)

    longitude = myd03.variables["Longitude"][:,:] # Reading Specific Variable 'Longitude'.
    #longitude = np.ravel(longitude) # Changing The Shape Of The Variable 'Longitude'.
    #longitude = np.array(longitude).byteswap().newbyteorder() # Addressing Byteswap For Big Endian Error.
    lon = np.dstack((lon,longitude))
    print('Latitude Shape Is: ',lon.shape)


    

Latitude Shape Is:  (2030, 1354, 2)
Latitude Shape Is:  (2030, 1354, 2)
Latitude Shape Is:  (2030, 1354, 3)
Latitude Shape Is:  (2030, 1354, 3)
Latitude Shape Is:  (2030, 1354, 4)
Latitude Shape Is:  (2030, 1354, 4)


In [38]:
lon.shape

(2030, 1354, 4)

In [37]:
latitude = np.ravel(latitude) # Changing The Shape Of The Variable 'Latitude'.
longitude = np.ravel(longitude) # Changing The Shape Of The Variable 'Longitude'.

In [38]:
lat = da.from_array(lat, chunks =(2030,1354,1))
lon = da.from_array(lon, chunks =(2030,1354,1))

In [39]:
cm.size

10994480

In [40]:
cm

dask.array<array, shape=(2030, 1354, 4), dtype=float32, chunksize=(2030, 1354, 1)>

In [41]:
lat.shape

(2030, 1354, 4)

In [42]:
MOD03_fn2

['MYD03.A2008001.0010.006.2012066122416.hdf',
 'MYD03.A2008001.0000.006.2012066122450.hdf',
 'MYD03.A2008001.0005.006.2012066122516.hdf']

In [43]:
lon.size

10994480

In [44]:
CMB123 = lat.astype(np.int8)
#CMB123 = xr.DataArray(CMB123)
CMB123 = da.from_array(CMB123, chunks =(2030,1354,1))
CMB124 = lon.astype(np.int8)
#CMB124 = xr.DataArray(CMB124)
CMB124 = da.from_array(CMB124, chunks =(2030,1354,1))

In [45]:
CMB123.compute()

array([[[ 0, 34, 69, 52],
        [ 0, 34, 69, 52],
        [ 0, 34, 69, 52],
        ...,
        [ 0, 30, 62, 47],
        [ 0, 30, 62, 47],
        [ 0, 30, 62, 47]],

       [[ 0, 34, 69, 52],
        [ 0, 34, 69, 52],
        [ 0, 34, 69, 52],
        ...,
        [ 0, 30, 62, 47],
        [ 0, 30, 62, 47],
        [ 0, 30, 62, 47]],

       [[ 0, 34, 69, 52],
        [ 0, 34, 69, 52],
        [ 0, 34, 69, 52],
        ...,
        [ 0, 30, 62, 47],
        [ 0, 30, 62, 47],
        [ 0, 30, 62, 47]],

       ...,

       [[ 0, 16, 52, 34],
        [ 0, 16, 52, 34],
        [ 0, 16, 52, 34],
        ...,
        [ 0, 13, 47, 30],
        [ 0, 13, 47, 30],
        [ 0, 13, 47, 30]],

       [[ 0, 16, 52, 34],
        [ 0, 16, 52, 34],
        [ 0, 16, 52, 34],
        ...,
        [ 0, 13, 47, 30],
        [ 0, 13, 47, 30],
        [ 0, 13, 47, 30]],

       [[ 0, 16, 52, 34],
        [ 0, 16, 52, 34],
        [ 0, 16, 52, 34],
        ...,
        [ 0, 13, 47, 30],
        [ 0, 13

In [46]:
dsa3 = xr.Dataset()
dsa3.coords['Latitude'] = (('x','y','z'), lat)
dsa3.coords['Longitude'] = (('x','y','z'), lon)
#dsa3.coords['Bit0'] = (('x','y','z'), cm1)
#dsa3.coords['Bit12'] = (('x','y','z'), cm)
dsa3.coords['LatInt'] = (('x','y','z'), CMB123)
dsa3.coords['LonInt'] = (('x','y','z'), CMB124)

In [47]:
dsa3

<xarray.Dataset>
Dimensions:    (x: 2030, y: 1354, z: 4)
Coordinates:
    Latitude   (x, y, z) float32 dask.array<shape=(2030, 1354, 4), chunksize=(2030, 1354, 1)>
    Longitude  (x, y, z) float32 dask.array<shape=(2030, 1354, 4), chunksize=(2030, 1354, 1)>
    LatInt     (x, y, z) int8 dask.array<shape=(2030, 1354, 4), chunksize=(2030, 1354, 1)>
    LonInt     (x, y, z) int8 dask.array<shape=(2030, 1354, 4), chunksize=(2030, 1354, 1)>
Dimensions without coordinates: x, y, z
Data variables:
    *empty*

In [48]:
cmr.size

10994480

In [49]:
CMrd = cmr.compute()
#CMrd = np.ravel(CMrd)

In [50]:
CMrd

array([[[   0.,  -13.,   49.,   49.],
        [   0.,  -13.,   49.,   49.],
        [   0.,  115.,   49.,   49.],
        ...,
        [   0., -111.,  -77.,  -15.],
        [   0., -111.,  -79.,  -15.],
        [   0., -111.,  -79., -111.]],

       [[   0.,  115.,   49.,   49.],
        [   0.,  115.,   49.,   51.],
        [   0.,  115.,   49.,   49.],
        ...,
        [   0., -111.,  -79.,  -15.],
        [   0., -111.,  -79.,  -15.],
        [   0., -111.,  -79.,  -15.]],

       [[   0.,  115.,   49.,   49.],
        [   0.,  115.,   49.,   51.],
        [   0.,  117.,   49.,   49.],
        ...,
        [   0., -111.,  -79.,  -15.],
        [   0., -111.,  -79.,  -15.],
        [   0., -111.,  -79.,  -15.]],

       ...,

       [[   0.,   49.,  -73.,  -13.],
        [   0.,   49.,  -73.,  -13.],
        [   0.,   49.,  -73.,  -13.],
        ...,
        [   0.,  -79.,  119., -111.],
        [   0.,  -79.,  119., -111.],
        [   0.,  -79.,  -73., -111.]],

       [[   0.,

In [51]:
dsa3['CloudFraction'] = (('x','y','z'), cm)

In [52]:
dsa3

<xarray.Dataset>
Dimensions:        (x: 2030, y: 1354, z: 4)
Coordinates:
    Latitude       (x, y, z) float32 dask.array<shape=(2030, 1354, 4), chunksize=(2030, 1354, 1)>
    Longitude      (x, y, z) float32 dask.array<shape=(2030, 1354, 4), chunksize=(2030, 1354, 1)>
    LatInt         (x, y, z) int8 dask.array<shape=(2030, 1354, 4), chunksize=(2030, 1354, 1)>
    LonInt         (x, y, z) int8 dask.array<shape=(2030, 1354, 4), chunksize=(2030, 1354, 1)>
Dimensions without coordinates: x, y, z
Data variables:
    CloudFraction  (x, y, z) float32 dask.array<shape=(2030, 1354, 4), chunksize=(2030, 1354, 1)>

In [54]:
from dask.distributed import Client

client = Client("tcp://10.48.180.20:33293")
client

Client Scheduler: tcp://10.48.180.20:33293 Dashboard: http://10.48.180.20:8787/status,Cluster Workers: 8 Cores: 40 Memory: 274.70 GB


In [53]:
#dsa3.Bit12.values

In [56]:
dsa4 = list(dsa3.groupby('LatInt', 'LonInt'))
%time

CPU times: user 10 µs, sys: 0 ns, total: 10 µs
Wall time: 27.9 µs


In [57]:
dsa4.index

<function list.index>

In [58]:
%time
dsa4

CPU times: user 9 µs, sys: 2 µs, total: 11 µs
Wall time: 21.5 µs


[(0, <xarray.Dataset>
  Dimensions:        (stacked_x_y_z: 2748620)
  Coordinates:
      Latitude       (stacked_x_y_z) float32 dask.array<shape=(2748620,), chunksize=(686478,)>
      Longitude      (stacked_x_y_z) float32 dask.array<shape=(2748620,), chunksize=(686478,)>
      LatInt         (stacked_x_y_z) int8 dask.array<shape=(2748620,), chunksize=(686478,)>
      LonInt         (stacked_x_y_z) int8 dask.array<shape=(2748620,), chunksize=(686478,)>
    * stacked_x_y_z  (stacked_x_y_z) MultiIndex
    - x              (stacked_x_y_z) int64 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
    - y              (stacked_x_y_z) int64 0 1 2 3 4 5 6 ... 23 24 25 26 27 28 29
    - z              (stacked_x_y_z) int64 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
  Data variables:
      CloudFraction  (stacked_x_y_z) float32 dask.array<shape=(2748620,), chunksize=(686478,)>),
 (13, <xarray.Dataset>
  Dimensions:        (stacked_x_y_z: 6702)
  Coordinates:
      Latitude       (stacked_x_y_z) float32 dask.a

In [59]:
futures = client.scatter(dsa4[0][1], broadcast=True)

In [60]:
dsa4[0][1].LatInt.values
%time

/srv/conda/lib/python3.6/site-packages/distributed/worker.py:2791: UserWarning: Large object of size 5.49 MB detected in task graph: 
  ("('reshape-b2b9a18ca6ac93c2bfee32e6976f4f23', 2)" ... 4, 2745908]),))
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
  % (format_bytes(len(b)), s))


CPU times: user 10 µs, sys: 2 µs, total: 12 µs
Wall time: 23.8 µs


In [97]:
cm

array([[[0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        ...,
        [0., 0., 1., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 1., 0., 0.],
        [0., 1., 0., 1.],
        [0., 1., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 1., 0., 0.],
        [0., 1., 0., 1.],
        [0., 2., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       ...,

       [[0., 0., 3., 1.],
        [0., 0., 3., 1.],
        [0., 0., 3., 1.],
        ...,
        [0., 0., 3., 0.],
        [0., 0., 3., 0.],
        [0., 0., 3., 0.]],

       [[0., 0., 3., 1.],
        [0., 2., 3., 1.],
        [0., 0., 3., 0.],
        ...,
        [0., 0., 3., 0.],
        [0., 0., 3., 0.],
        [0., 0., 3., 0.]],

       [[0., 1., 3., 1.],
        [0., 2., 3., 1.],
        [0., 2., 3., 0.],
        ...,
        [0., 0., 1., 0.],
        [0., 0.

In [61]:
dsa4[1][1].LatInt.values
%time

CPU times: user 9 µs, sys: 1e+03 ns, total: 10 µs
Wall time: 21.7 µs


In [62]:
list1=[]
for key, value in dsa4:
    #print(key)
    list1.append(key)

In [63]:
Nobs = np.sum(cm1.compute()>=0)
print("Number Of Total Pixel: ", np.sum(cm1.compute()>=0), " Pixels")
print("Number Of Pixel With Cloud Mask Determined: ", np.sum(cm1.compute()==1), " Pixels")

Number Of Total Pixel:  10994480  Pixels
Number Of Pixel With Cloud Mask Determined:  8245860  Pixels


In [64]:
lat_index = value_locate(lat_bnd,latitude)
lon_index = value_locate(lon_bnd,longitude)
latlon_index = lat_index*nlon + lon_index

In [65]:
latlon_index

array([51315, 51315, 51315, ..., 43418, 43418, 43418])

In [68]:
np.unique(latlon_index)

array([43416, 43417, 43418, 43772, 43773, 43774, 43775, 43776, 43777,
       43778, 43779, 44126, 44127, 44128, 44129, 44130, 44131, 44132,
       44133, 44134, 44135, 44136, 44137, 44138, 44139, 44477, 44478,
       44479, 44480, 44481, 44482, 44483, 44484, 44485, 44486, 44487,
       44488, 44489, 44490, 44491, 44492, 44493, 44494, 44495, 44496,
       44497, 44498, 44499, 44834, 44835, 44836, 44837, 44838, 44839,
       44840, 44841, 44842, 44843, 44844, 44845, 44846, 44847, 44848,
       44849, 44850, 44851, 44852, 44853, 44854, 44855, 44856, 44857,
       44858, 44859, 44860, 45194, 45195, 45196, 45197, 45198, 45199,
       45200, 45201, 45202, 45203, 45204, 45205, 45206, 45207, 45208,
       45209, 45210, 45211, 45212, 45213, 45214, 45215, 45216, 45217,
       45218, 45219, 45220, 45554, 45555, 45556, 45557, 45558, 45559,
       45560, 45561, 45562, 45563, 45564, 45565, 45566, 45567, 45568,
       45569, 45570, 45571, 45572, 45573, 45574, 45575, 45576, 45577,
       45578, 45579,

In [69]:
print('computing simple level3 statistics')
latlon_index_unique = np.unique(latlon_index)
print('this granule occupies',latlon_index_unique.size,'1x1 degree box')

computing simple level3 statistics
this granule occupies 576 1x1 degree box


In [70]:
for i in np.arange(latlon_index_unique.size):
    j=latlon_index_unique[i]
    TOT_pix[j] = TOT_pix[j]+np.sum(CM[np.where(latlon_index == j)]>=0)
    CLD_pix[j] = CLD_pix[j]+np.sum(CM[np.where(latlon_index == j)]<=1) 

IndexError: index 2748584 is out of bounds for axis 0 with size 2030

In [73]:
cm = CM.ravel()
for i in np.arange(latlon_index_unique.size):
    j=latlon_index_unique[i]
    TOT_pix[j] = TOT_pix[j]+np.sum(cm[np.where(latlon_index == j)]>=0)
    CLD_pix[j] = CLD_pix[j]+np.sum(cm[np.where(latlon_index == j)]<=1) 

In [74]:
print('derive the averaged Level-3 cloud fraction')
total_cloud_fraction  =  division(CLD_pix,TOT_pix).reshape([nlat,nlon])
print(np.nansum(total_cloud_fraction))

derive the averaged Level-3 cloud fraction
400.23007086613126


In [ ]:
lat = lat.compute()
lon = lon.compute()
CMB123 = CMB123.compute()
CMB124 = CMB124.compute()
#cm = cm.compute()

dsa7 = xr.Dataset()
dsa7.coords['Latitude'] = (('x','y','z'), lat)
dsa7.coords['Longitude'] = (('x','y','z'), lon)
#dsa3.coords['Bit0'] = (('x','y','z'), cm1)
#dsa3.coords['Bit12'] = (('x','y','z'), cm)
dsa7.coords['LatInt'] = (('x','y','z'), CMB123)
dsa7.coords['LonInt'] = (('x','y','z'), CMB124)
#dsa3['CloudFraction'] = (('x','y','z'), cm)


In [76]:
dsa7

<xarray.Dataset>
Dimensions:    (x: 2030, y: 1354, z: 4)
Coordinates:
    Latitude   (x, y, z) float32 0.0 34.241398 69.99062 ... 47.45127 30.623335
    Longitude  (x, y, z) float32 0.0 14.006025 15.142107 ... 47.78136 38.475307
    LatInt     (x, y, z) int8 0 34 69 52 0 34 69 52 0 ... 0 13 47 30 0 13 47 30
    LonInt     (x, y, z) int8 0 14 15 15 0 14 15 15 0 ... 0 32 47 38 0 32 47 38
Dimensions without coordinates: x, y, z
Data variables:
    *empty*

In [98]:
cm.shape

(2030, 1354, 4)

In [99]:
CM.shape

(2030, 1354)

In [101]:
cmravel= CM.ravel()

In [102]:
cmravel.shape

(2748620,)

In [103]:
cmravelda = da.from_array(cmravel, chunks =(2030,))

In [120]:
cm12 = cmravelda.mean
cm123 = cmravelda.compute()
cm123.size

2748620

In [92]:
cm = cm.compute()
dsa7['CloudFraction'] = (('x','y','z'), cm)

In [93]:
dsa7

<xarray.Dataset>
Dimensions:        (x: 2030, y: 1354, z: 4)
Coordinates:
    Latitude       (x, y, z) float32 0.0 34.241398 ... 47.45127 30.623335
    Longitude      (x, y, z) float32 0.0 14.006025 ... 47.78136 38.475307
    LatInt         (x, y, z) int8 0 34 69 52 0 34 69 52 ... 13 47 30 0 13 47 30
    LonInt         (x, y, z) int8 0 14 15 15 0 14 15 15 ... 32 47 38 0 32 47 38
Dimensions without coordinates: x, y, z
Data variables:
    CloudFraction  (x, y, z) float32 0.0 1.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 3.0 0.0

In [96]:
i = 0
while i < len(list1):
    #print(list1[i])
    a = list1[i]
    #print(dsa4[i])
    dsa8 =  list(dsa7.groupby('LatInt', 'LonInt'))
    #dsa81 = dsa8[0][1]
    #dsa91 = list(dsa81.groupby('LonInt'))
    #dsa61 = dsa51[0][1] 
    #dsa71 = list(dsa61.groupby('Bit0', 'Bit12'))
    #dsa81 = dsa71[0][1]
    #dsa91 = list(dsa81.groupby('Bit12'))
    #dsa81 = dsa71[0][1]
    #dsa91 = dsa81.groupby('Bit12').count()/Nobs*100
    #print(dsa91)
    print(dsa8)
    i += 1 
%time    

[(0, <xarray.Dataset>
Dimensions:        (stacked_x_y_z: 2748620)
Coordinates:
    Latitude       (stacked_x_y_z) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    Longitude      (stacked_x_y_z) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    LatInt         (stacked_x_y_z) int8 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
    LonInt         (stacked_x_y_z) int8 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
  * stacked_x_y_z  (stacked_x_y_z) MultiIndex
  - x              (stacked_x_y_z) int64 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
  - y              (stacked_x_y_z) int64 0 1 2 3 4 5 6 ... 23 24 25 26 27 28 29
  - z              (stacked_x_y_z) int64 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
Data variables:
    CloudFraction  (stacked_x_y_z) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0), (13, <xarray.Dataset>
Dimensions:        (stacked_x_y_z: 6702)
Coordinates:
    Latitude       (stacked_x_y_z) float32 13.998945 13.9921665 ... 13.128265
    Longitude      (stacked_x_y_z) float32 32.928566 32.87365 ..

KeyboardInterrupt: 

In [120]:
%time
i = 0
start_time = time.time()
while i < len(list1):
    #print(list1[i])
    a = list1[i]
    #print(dsa4[i])
    dsa4 =  list(dsa3.groupby('LatInt', 'LonInt'))
    dsa41 = dsa4[i][1]
    dsa51 = list(dsa41.groupby('LonInt'))
    dsa61 = dsa51[0][1] 
    dsa71 = list(dsa61.groupby('Bit0', 'Bit12'))
    dsa81 = dsa71[0][1]
    dsa91 = list(dsa81.groupby('Bit12'))
    dsa81 = dsa71[0][1]
    dsa91 = dsa81.groupby('Bit12').count()
    print(dsa91)
    i += 1
end_time = time.time()
print("Total Time Taken This Loop: ", end_time - start_time)
hours, rem = divmod(end_time-start_time, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
%time

CPU times: user 3 µs, sys: 3 µs, total: 6 µs
Wall time: 15 µs


/Users/saviosebastian/anaconda3/lib/python3.6/site-packages/xarray/core/groupby.py:723: FutureWarning: Default reduction dimension will be changed to the grouped dimension after xarray 0.12. To silence this warning, pass dim=xarray.ALL_DIMS explicitly.
  **kwargs)


<xarray.Dataset>
Dimensions:        (Bit12: 1)
Coordinates:
  * Bit12          (Bit12) int64 2
Data variables:
    CloudFraction  (Bit12) int64 1
<xarray.Dataset>
Dimensions:        (Bit12: 1)
Coordinates:
  * Bit12          (Bit12) int64 3
Data variables:
    CloudFraction  (Bit12) int64 372
<xarray.Dataset>
Dimensions:        (Bit12: 2)
Coordinates:
  * Bit12          (Bit12) int64 2 3
Data variables:
    CloudFraction  (Bit12) int64 1 24
<xarray.Dataset>
Dimensions:        (Bit12: 1)
Coordinates:
  * Bit12          (Bit12) int64 3
Data variables:
    CloudFraction  (Bit12) int64 12
<xarray.Dataset>
Dimensions:        (Bit12: 4)
Coordinates:
  * Bit12          (Bit12) int64 0 1 2 3
Data variables:
    CloudFraction  (Bit12) int64 2 46 49 1121
<xarray.Dataset>
Dimensions:        (Bit12: 3)
Coordinates:
  * Bit12          (Bit12) int64 1 2 3
Data variables:
    CloudFraction  (Bit12) int64 2 36 1150
<xarray.Dataset>
Dimensions:        (Bit12: 4)
Coordinates:
  * Bit12          (Bit12) 

In [ ]:
%time
i = 0
while i < len(list1):
    #print(list1[i])
    a = list1[i]
    #print(dsa4[i])
    dsa4 =  list(dsa3.groupby('LatInt', 'LonInt'))
    dsa41 = dsa4[i][1]
    dsa51 = list(dsa41.groupby('LonInt'))
    dsa61 = dsa51[0][1] 
    dsa71 = list(dsa61.groupby('Bit0', 'Bit12'))
    dsa81 = dsa71[0][1]
    dsa91 = list(dsa81.groupby('Bit12'))
    dsa81 = dsa71[0][1]
    dsa91 = dsa81.groupby('Bit12')
    print(dsa91)
    i += 1

In [198]:
time.time()

1546942482.3784

In [117]:
%time
start_time = time.time()
i = 0

while i < len(list1):
    #print(list1[i])
    a = list1[i]
    #print(dsa4[i])
    dsa4 =  list(dsa3.groupby('LatInt', 'LonInt'))
    dsa41 = dsa4[i][1]
    dsa51 = list(dsa41.groupby('LonInt'))
    dsa61 = dsa51[0][1] 
    dsa71 = list(dsa61.groupby('Bit0', 'Bit12'))
    dsa81 = dsa71[0][1]
    dsa91 = list(dsa81.groupby('Bit12'))
    dsa81 = dsa71[0][1]
    dsa91 = list(dsa81.groupby('Bit12'))
    print(dsa91)
    i += 1

end_time = time.time()
print("Total Time Taken This Loop: ", end_time - start_time)
hours, rem = divmod(end_time-start_time, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs
[(2, <xarray.Dataset>
Dimensions:        (x: 1)
Coordinates:
    Latitude       (x) float64 2.0
    Longitude      (x) float64 2.0
    Bit0           (x) int64 2
    Bit12          (x) int64 dask.array<shape=(1,), chunksize=(1,)>
    LatInt         (x) int8 2
    LonInt         (x) int8 2
Dimensions without coordinates: x
Data variables:
    CloudFraction  (x) int64 2)]
[(3, <xarray.Dataset>
Dimensions:        (x: 372)
Coordinates:
    Latitude       (x) float64 14.0 13.99 13.99 14.0 ... 13.84 13.84 13.83 13.83
    Longitude      (x) float64 28.95 28.97 28.99 28.88 ... 28.94 28.96 28.98
    Bit0           (x) int64 1 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1 1
    Bit12          (x) int64 dask.array<shape=(372,), chunksize=(372,)>
    LatInt         (x) int8 13 13 13 13 13 13 13 13 ... 13 13 13 13 13 13 13 13
    LonInt         (x) int8 28 28 28 28 28 28 28 28 ... 28 28 28 28 28 28 28 28
Dimensions without coordinate

In [118]:
from dask.distributed import Client
client = Client()

In [119]:
client.cluster

In [122]:
from dask.distributed import Client

# Setup a local cluster.
# By default this sets up 1 worker per core
client = Client()
client

Client Scheduler: tcp://127.0.0.1:54707,Cluster Workers: 4 Cores: 4 Memory: 17.18 GB


In [123]:
%time
start_time = time.time()
i = 0

while i < len(list1):
    #print(list1[i])
    a = list1[i]
    #print(dsa4[i])
    dsa4 =  list(dsa3.groupby('LatInt', 'LonInt'))
    dsa41 = dsa4[i][1]
    dsa51 = list(dsa41.groupby('LonInt'))
    dsa61 = dsa51[0][1] 
    dsa71 = list(dsa61.groupby('Bit0', 'Bit12'))
    dsa81 = dsa71[0][1]
    dsa91 = list(dsa81.groupby('Bit12'))
    dsa81 = dsa71[0][1]
    dsa91 = list(dsa81.groupby('Bit12'))
    print(dsa91)
    i += 1

end_time = time.time()
print("Total Time Taken This Loop: ", end_time - start_time)
hours, rem = divmod(end_time-start_time, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 11 µs
[(2, <xarray.Dataset>
Dimensions:        (x: 1)
Coordinates:
    Latitude       (x) float64 2.0
    Longitude      (x) float64 2.0
    Bit0           (x) int64 2
    Bit12          (x) int64 dask.array<shape=(1,), chunksize=(1,)>
    LatInt         (x) int8 2
    LonInt         (x) int8 2
Dimensions without coordinates: x
Data variables:
    CloudFraction  (x) int64 2)]
[(3, <xarray.Dataset>
Dimensions:        (x: 372)
Coordinates:
    Latitude       (x) float64 14.0 13.99 13.99 14.0 ... 13.84 13.84 13.83 13.83
    Longitude      (x) float64 28.95 28.97 28.99 28.88 ... 28.94 28.96 28.98
    Bit0           (x) int64 1 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1 1
    Bit12          (x) int64 dask.array<shape=(372,), chunksize=(372,)>
    LatInt         (x) int8 13 13 13 13 13 13 13 13 ... 13 13 13 13 13 13 13 13
    LonInt         (x) int8 28 28 28 28 28 28 28 28 ... 28 28 28 28 28 28 28 28
Dimensions without coordina

In [30]:
from dask.distributed import Client
client = Client()
client

Client Scheduler: tcp://127.0.0.1:54904 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 17.18 GB
